In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest

In [2]:
import pandas as pd

dfBen = pd.read_excel("../shared_data_read_only/Data/Article1/Hackaton_Benevoles_JPMORGAN.xlsx")
dfBin = pd.read_excel("../shared_data_read_only/Data/Article1/Hackaton_Binomes_JPMORGAN.xlsx")
dfJeu = pd.read_excel("../shared_data_read_only/Data/Article1/Hackaton_Jeunes_JPMORGAN.xlsx")

df = pd.read_csv("../shared_documents/pairs_cleaned.csv")

/tmp/ipykernel_95275/3606502650.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../shared_documents/pairs_cleaned.csv")


In [3]:
df['y_true'] = df['binome_statut'].isin(['COMPLETED', 'ACTIVE']).astype(int)

In [4]:
df_work = df.copy()

# 1) registration_date -> days_since_registration
import pandas as pd, numpy as np
df_work["registration_date"] = pd.to_datetime(df_work["registration_date"], errors="coerce")
ref_date = pd.Timestamp.today().normalize()
df_work["days_since_registration"] = (ref_date - df_work["registration_date"]).dt.days
num_cols = ["average_grade","engagement_score","days_since_registration"]
cat_cols = ["workfield","field_of_study","study_level","degree","needs",
            "program","desired_exchange_frequency","binome_statut"]
id_cols  = [c for c in ["binome_id","mentor_id","mentee_id"] if c in df_work.columns]


KeyError: 'registration_date'

In [ ]:
cols = [
    'binome_statut', 'binome_acceptance_delay', 'binome_date_update_statut',
    'workfield', 'current_role', 'needs_to_address_mentor',
    'field_of_study', 'study_level', 'degree', 'needs_to_address_mentee',
    'average_grade', 'program', 'engagement_score',
    'registration_date_mentee', 'desired_exchange_frequency', 'hobby',
    'project_confidence_level', 'project_development_level',
    'binome_score_clean'
]
X = df[cols].copy()

X = X.fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.select_dtypes(include=['float64', 'int64']))

# Suppose we want to keep top 5 PCs
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

print("Explained variance ratio:", pca.explained_variance_ratio_)
print(f"Cumulative variance: {pca.explained_variance_ratio_.sum()*100:.2f}%")

# Combine into a DataFrame
pca_cols = [f"PC{i}" for i in range(1, 4)]
df_pca_5 = pd.DataFrame(X_pca, columns=pca_cols)

# Optionally attach to your main dataframe
df_out = pd.concat([df.reset_index(drop=True), df_pca_5], axis=1)


In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

# Fit PCA with 5 components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

print("Explained variance ratio:", pca.explained_variance_ratio_)
print(f"Cumulative variance: {pca.explained_variance_ratio_.sum()*100:.2f}%")

# Combine PCA results into a DataFrame
pca_cols = [f"PC{i}" for i in range(1, 4)]
df_pca_5 = pd.DataFrame(X_pca, columns=pca_cols)

# Optionally attach to your main dataframe
df_out = pd.concat([df.reset_index(drop=True), df_pca_5], axis=1)

# Save outputs
df_out.to_csv("pca_5components_output.csv", index=False)

print("✅ PCA results saved to 'pca_5components_output.csv'")
print(df_pca_5.head())


In [ ]:
import pandas as pd

feature_names = X.select_dtypes(include=['float64','int64']).columns

loadings = pd.DataFrame(
    pca.components_.T,
    index=feature_names,
    columns=[f'PC{i+1}' for i in range(pca.n_components_)]
)

TOP = 5
for pc in loadings.columns:
    print(f"\nTop {TOP} features driving {pc}:")
    print(loadings[pc].abs().sort_values(ascending=False).head(TOP))

In [ ]:
import numpy as np
import pandas as pd

def top5_mentors_for_mentee(
    mentee_row: pd.Series,
    dfMentors: pd.DataFrame,
    rf_model,
    a_post: float,
    b_post: float,
    scaler,
    feature_cols: list,
    num_cols: list,
    cat_cols: list,
    mentor_id_col: str = "mentor_id"
) -> pd.DataFrame:
    """
    Given a mentee row, score all mentors and return top-5 by calibrated success probability.
    
    mentee_row: Single mentee (pd.Series) with raw columns (same schema as training)
    dfMentors: All mentors (pd.DataFrame)
    rf_model: Trained RandomForestClassifier
    a_post, b_post: Posterior means from Bayesian calibration (logit calibration)
    scaler: Fitted StandardScaler for numeric columns
    feature_cols: Final feature columns used in training (order matters)
    num_cols: Numeric feature list used in training (after prefixing if you used prefixes)
    cat_cols: Categorical columns to one-hot encode (raw names before prefixing)
    mentor_id_col: Column name that identifies mentors in dfMentors
    
    Returns: DataFrame with [mentor_id, prob] sorted descending, top-5 rows.
    """
    # --- 1) Prefix and broadcast mentee features to all mentors ---
    mentee_pref = mentee_row.add_prefix("mentee_")
    mentors_pref = dfMentors.add_prefix("mentor_").copy()
    
    # Repeat mentee fields to match mentors' length
    mentee_block = pd.DataFrame([mentee_pref.values] * len(mentors_pref), columns=mentee_pref.index)
    pair_df = pd.concat([mentors_pref.reset_index(drop=True), mentee_block.reset_index(drop=True)], axis=1)

    # --- 2) Build feature matrix exactly like in training ---
    # Ensure numeric columns exist and are numeric
    for c in num_cols:
        if c not in pair_df:
            pair_df[c] = 0.0
        pair_df[c] = pd.to_numeric(pair_df[c], errors='coerce').fillna(0.0)

    # One-hot encode categorical cols (both mentor_* and mentee_* variants may exist)
    # cat_cols are raw names; we expand to prefixed forms if present
    expanded_cats = []
    for raw in cat_cols:
        mc = "mentor_" + raw
        tc = "mentee_" + raw
        if mc in pair_df.columns: expanded_cats.append(mc)
        if tc in pair_df.columns: expanded_cats.append(tc)

    if expanded_cats:
        pair_df = pd.get_dummies(pair_df, columns=expanded_cats, drop_first=True)

    # Add any missing columns expected by the model; keep exact order
    for col in feature_cols:
        if col not in pair_df:
            pair_df[col] = 0
    pair_X = pair_df[feature_cols].copy()

    # Standardize numeric columns using the training scaler
    # (Assumes scaler was fit only on num_cols)
    pair_X[num_cols] = scaler.transform(pair_X[num_cols])

    # --- 3) Random Forest probability for each mentor ---
    rf_prob = rf_model.predict_proba(pair_X)[:, 1]
    rf_logit = np.log(np.clip(rf_prob, 1e-6, 1-1e-6) / np.clip(1-rf_prob, 1e-6, 1-1e-6))

    # --- 4) Bayesian calibration (Platt with posterior means) ---
    prob_cal = 1.0 / (1.0 + np.exp(-(a_post + b_post * rf_logit)))

    # --- 5) Return top-5 mentors with probabilities ---
    out = pd.DataFrame({
        "mentor_id": dfMentors[mentor_id_col].values,
        "prob": prob_cal
    }).sort_values("prob", ascending=False).head(5).reset_index(drop=True)

    return out


In [ ]:
mentee_row = dfJeu.iloc[0]
mentee_row

In [ ]:
# Temporary example values (replace with real posterior means)
a_post = -0.2
b_post = 0.9


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = (
    dfBin.merge(dfJeu, on="mentee_id", how="left")
         .merge(dfBen, on="mentor_id", how="left")
)

num_cols = ['binome_score', 'binome_acceptance_delay',
            'engagement_score', 'project_confidence_level', 'average_grade']
cat_cols = ['workfield', 'current_role', 'field_of_study', 'needs']

In [ ]:
top5 = top5_mentors_for_mentee(
    mentee_row = mentee_row,       # one mentee row
    dfMentors  = dfBen,            # full mentor dataframe
    rf_model   = rf,               # trained RandomForest
    a_post     = a_post,           # Bayesian calibration intercept
    b_post     = b_post,           # Bayesian calibration slope
    scaler     = scaler,           # StandardScaler used in training
    feature_cols = X_rf.columns,   # same feature order as training
    num_cols   = [
        'mentor_engagement_score',
        'mentee_engagement_score',
        'mentee_project_confidence_level',
        'mentee_project_development_level'
    ],
    cat_cols   = [
        'study_level',
        'field_of_study',
        'workfield',
        'degree',
        'program',
        'hobby'
    ],
    mentor_id_col = "mentor_id"
)

print(top5)
